# Random forest importance

In [1]:
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r feature_names

In [2]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
import matplotlib.pyplot as plt

# Reshape data
# Data ban đầu X(13166, 20, 128), y(13166)

# Reshape X về dạng 2d => X_2d(263320, 128), giữ nguyên 128 và đó là các đặc trưng
X_2d = X.reshape(-1, X.shape[-1])
# Reshape y thành y_reshaped(263320) bằng cách lặp mỗi phần tử trong y 20 lần
y_reshaped = np.repeat(y, 20)

In [4]:
# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=500)
# Nó chỉ định số lượng cây quyết định sẽ được xây dựng và kết hợp để tạo thành khu rừng ngẫu nhiên trong thuật toán random forest.

In [5]:
# Huấn luyện mô hình
model.fit(X_2d, y_reshaped)

RandomForestClassifier(n_estimators=500)

In [6]:
# Lấy tầm quan trọng của đặc trưng
feature_importances = model.feature_importances_

# In tầm quan trọng của đặc trưng
print(feature_importances)

[0.0173191  0.01329185 0.01332595 0.01295527 0.01310233 0.01310117
 0.01292544 0.01320068 0.01287461 0.01282813 0.01272915 0.0127903
 0.01862665 0.03283118 0.02992361 0.02794496 0.02486339 0.03638818
 0.0263802  0.02512459 0.01594859 0.01960635 0.01651946 0.01815531
 0.01569521 0.0140855  0.01449727 0.01665159 0.01190635 0.00954217
 0.00825479 0.00527411 0.00549142 0.00496272 0.00464538 0.00521099
 0.00427064 0.00489688 0.00394844 0.00514944 0.00448527 0.00436076
 0.00472488 0.00535518 0.00465122 0.00403647 0.00419314 0.00491528
 0.00404518 0.00469305 0.00413749 0.00416678 0.00579511 0.00543212
 0.00498129 0.00533605 0.00567071 0.00700769 0.00578696 0.00556822
 0.00856659 0.0068113  0.00672132 0.00513189 0.00490619 0.00543885
 0.00490035 0.00462876 0.00432527 0.00474407 0.00455251 0.00538707
 0.00480095 0.00524356 0.00545892 0.00412993 0.00404477 0.00448987
 0.0052109  0.00597181 0.00552425 0.00546457 0.005085   0.00502908
 0.00493184 0.00459162 0.00368623 0.0038131  0.00356125 0.00374

In [7]:
feature_importances.shape

(128,)

In [8]:
# Tạo dataframe gồm tên đặc trưng và tầm quan trọng
feature_importances_df = pd.DataFrame({"feature_name": feature_names, "importance": feature_importances})

# Sắp xếp dataframe theo tầm quan trọng
feature_importances_df = feature_importances_df.sort_values("importance", ascending=False)

# Lấy index của dataframe
indices = feature_importances_df.index

# In index
print(indices)

Index([ 17,  13,  14,  15,  18,  19,  16,  21,  12,  23,
       ...
        89, 107, 115, 109,  86, 114, 112,  90, 106,  88],
      dtype='int64', length=128)


In [9]:
# Xác định số đặc trưng sẽ giữ lại để train  model
k = 64
top_k_indices = indices[:k]

In [10]:
# Giữ lại k đặc trưng tốt nhất trong tệp dữ liệu ban đầu
selected_X = X[:,:,top_k_indices]
selected_X_test = X_test[:,:,top_k_indices]

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(20, k, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 64, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 64, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 32, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 32, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 32, 64)        51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 32, 64)        102464    
                                                     

In [12]:
model.fit(selected_X, y, epochs=10, batch_size=16)

Epoch 1/10


823/823 [==============================] - 32s 37ms/step - loss: 0.8632 - accuracy: 0.6561
Epoch 2/10
823/823 [==============================] - 31s 37ms/step - loss: 0.4598 - accuracy: 0.8418
Epoch 3/10
823/823 [==============================] - 33s 40ms/step - loss: 0.3663 - accuracy: 0.8725
Epoch 4/10
823/823 [==============================] - 33s 40ms/step - loss: 0.3168 - accuracy: 0.8846
Epoch 5/10
823/823 [==============================] - 33s 40ms/step - loss: 0.3016 - accuracy: 0.8918
Epoch 6/10
823/823 [==============================] - 33s 40ms/step - loss: 0.2830 - accuracy: 0.8998
Epoch 7/10
823/823 [==============================] - 33s 40ms/step - loss: 0.2614 - accuracy: 0.9092
Epoch 8/10
823/823 [==============================] - 32s 39ms/step - loss: 0.2514 - accuracy: 0.9120
Epoch 9/10
823/823 [==============================] - 30s 37ms/step - loss: 0.2320 - accuracy: 0.9151
Epoch 10/10
823/823 [==============================] - 31s 37ms/step - loss: 0.2

In [13]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 8s 19ms/step - loss: 0.1597 - accuracy: 0.9387
Độ chính xác trên tập kiểm tra: 0.9387057423591614
412/412 [==============================] - 8s 19ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1986   16   11   21]
 [   0   18 2637    4  149]
 [   0  336    7 1392    5]
 [   0   20  220    0 3502]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.84      0.98      0.90      2034
           2       0.92      0.94      0.93      2808
           3       0.99      0.80      0.88      1740
           4       0.95      0.94      0.94      3742

    accuracy                           0.94     13166
   macro avg       0.94      0.93      0.93     13166
weighted avg       0.94      0.94      0.94     13166

